In [1]:
from src.utils.spark_builder import get_spark_session

spark = get_spark_session("Jupyter Test")
print("!! SparkSession 생성 완료!")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9af4712b-8a79-4ddd-809d-600fe22dbc6c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.367 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.3 in central
	found org.apache.kafka#kafka-clients;3.

!! SparkSession 생성 완료!


In [2]:
silver_path = "s3a://silver/gdelt_events"
try:
    df_gdelt_silver = spark.read.format("delta").load(silver_path)
    print(f"GDELT silver 데이터 로드 성공: {silver_path}")

    # 기본 정보 출력
    print(f"총 레코드 수 : {df_gdelt_silver.count()}")
    print(f"컬럼 수: {len(df_gdelt_silver.columns)}")

    # 스키마 정보
    print("\n 테이블 스키마: ")
    df_gdelt_silver.printSchema()

    # 상위 20개
    print("\n 상위 20개 레코드")
    df_gdelt_silver.show(20, truncate=False)
except Exception as e:
    print(f"데이터 로드 실패: {e}")
    print("1. MinIo에 Silver 버킷이 존재하는지")
    print("2. gdelt_events 폴더에 Delta 파일이 있는지")
    print("3. 데이터 파이프라인이 실행되었는지")

25/08/25 13:28:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


GDELT silver 데이터 로드 성공: s3a://silver/gdelt_events


25/08/25 13:29:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


총 레코드 수 : 1276725
컬럼 수: 20

 테이블 스키마: 
root
 |-- event_date: string (nullable = true)
 |-- Actor1CountryCode: string (nullable = true)
 |-- EventRootCode: string (nullable = true)
 |-- day_of_week: long (nullable = true)
 |-- event_count: long (nullable = true)
 |-- avg_conflict_score: double (nullable = true)
 |-- stddev_conflict_score: double (nullable = true)
 |-- avg_tone: double (nullable = true)
 |-- unique_source_count: long (nullable = true)
 |-- GlobalEventID: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- MonthYear: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- FractionDate: string (nullable = true)
 |-- Actor1Code: string (nullable = true)
 |-- Actor1Name: string (nullable = true)
 |-- Actor1KnownGroupCode: string (nullable = true)
 |-- Actor1EthnicCode: string (nullable = true)
 |-- AvgTone: string (nullable = true)
 |-- QuadClass: string (nullable = true)


 상위 20개 레코드


+----------+-----------------+-------------+-----------+-----------+------------------+---------------------+--------+-------------------+-------------+--------+---------+----+------------+----------+--------------+--------------------+----------------+-------+---------+
|event_date|Actor1CountryCode|EventRootCode|day_of_week|event_count|avg_conflict_score|stddev_conflict_score|avg_tone|unique_source_count|GlobalEventID|Day     |MonthYear|Year|FractionDate|Actor1Code|Actor1Name    |Actor1KnownGroupCode|Actor1EthnicCode|AvgTone|QuadClass|
+----------+-----------------+-------------+-----------+-----------+------------------+---------------------+--------+-------------------+-------------+--------+---------+----+------------+----------+--------------+--------------------+----------------+-------+---------+
|null      |                 |             |null       |null       |null              |null                 |null    |null               |1259429063   |20240825|202408   |2024|2024.643

In [8]:
spark.stop()